## Schrodinger FEP+ dataset formatting

### Inputs
- ```.pdb``` file containing protein structures

- ```.sdf``` file containing ligand names, coordinates and DeltaG values


### Output

- ```.pdb``` file for each ligand

- Train, validation and test data files for aescore with the following format:

        DeltaG path/to/protein.pdb path/to/ligand.pdb




In [1]:
import rdkit
from rdkit import Chem
import os
import pickle
import pandas as pd
import numpy as np

In [2]:
def getDirNames(path):
    return [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]

In [3]:
def getLigandDataFromSDF(sdf_file, id_prop, dG_prop, removeHs=False):
    """
    Read in a SDF file and return a nested dictionary of ligand data.

    Parameters
    ----------
    sdf_file : str 
        Path to SDF file.
    id_prop : str
        Name of property in SDF file that contains the ligand ID.
    dG_prop : str
        Name of property in SDF file that contains the ligand binding free energy.
    removeHs : bool, optional
        Remove hydrogens from the ligand molecules. Default is False.
    """

    suppl = Chem.SDMolSupplier(sdf_file,removeHs=removeHs)
    lig_objs = [m for m in suppl if m is not None]
    lig_ids = [m.GetProp(id_prop) for m in lig_objs]
    lig_bfe = [float(m.GetProp(dG_prop)) for m in lig_objs]

    index_dict = {lig_id: index for index, lig_id in enumerate(lig_ids)}
    lig_data = {lig_id: {'dG': lig_bfe[index_dict[lig_id]], 'rdmol': lig_objs[index_dict[lig_id]]} for lig_id in lig_ids}

    return lig_data

In [4]:
def molToPDB(lig_data, target_name, outdir):
    """
    Write out a PDB file for each molecule in a dictionary of molecule data. 
    """
    saved = total = 0

    for lig_id in lig_data.keys():
        mol = lig_data[lig_id]['rdmol']
        try:
            Chem.MolToPDBFile(mol, '{}/{}_ligand_{}.pdb'.format(outdir, target_name, lig_id))
            saved += 1
        except:
            print('Error writing {} to PDB file'.format(lig_id))
        
        total += 1
    
    print('{} out of {} {} ligands saved to {}'.format(saved, total, target_name, outdir))
        

In [11]:
targets = getDirNames('../data')
mol_data_dict = {target: getLigandDataFromSDF('../data/{}/{}_ligands.sdf'.format(target,target), '_Name', 'r_user_dG.exp') for target in targets}

for target in mol_data_dict:
    molToPDB(mol_data_dict[target], target, '../data/{}'.format(target))

pickle.dump(mol_data_dict, open('../data/all_ligands_mol_dict.pkl', 'wb'))

21 out of 21 Jnk1 ligands saved to ../data/Jnk1
36 out of 36 Bace ligands saved to ../data/Bace
34 out of 34 p38 ligands saved to ../data/p38
23 out of 23 PTP1B ligands saved to ../data/PTP1B
42 out of 42 MCL1 ligands saved to ../data/MCL1
16 out of 16 Tyk2 ligands saved to ../data/Tyk2
11 out of 11 Thrombin ligands saved to ../data/Thrombin
16 out of 16 CDK2 ligands saved to ../data/CDK2


In [14]:
mol_dict = pickle.load(open('../data/all_ligands_mol_dict.pkl', 'rb'))

loc_dict = dict()

for target in mol_dict.keys():
    
    target_loc = '../data/{}/{}_protein.pdb'.format(target, target)
    for ligand in mol_dict[target]:

        dg = mol_dict[target][ligand]['dG']
        ligand_loc = '../data/{}/{}_ligand_{}.pdb'.format(target, target, ligand)
        entry = {'target':target, 'target_loc':target_loc, 'ligand':ligand, 'ligand_loc':ligand_loc, 'dG':dg}
        loc_dict['{}_{}'.format(target,ligand)] = entry

loc_df = pd.DataFrame.from_dict(data=loc_dict, orient='index')
pickle.dump(loc_df, open('../data/all_ligands_loc_df.pkl', 'wb'))


In [10]:
def stratifiedSample(df, col, n_samples):
    """
    Draw a sample from a dataframe, such that the sample is stratified based on the frequency of values in a column.
    """
    
    return df.groupby(col, group_keys=False).apply(lambda x: x.sample(int(np.rint(n_samples*len(x)/len(df))))).sample(frac=1)


In [11]:
loc_df = pickle.load(open('../data/all_ligands_loc_df.pkl', 'rb'))
loc_df = loc_df.round({'dG': 2})

val_set = stratifiedSample(loc_df, 'target', 20)
test_set = stratifiedSample(loc_df[~loc_df.index.isin(val_set.index)], 'target', 20)
train_set = loc_df[~loc_df.index.isin(val_set.index) & ~loc_df.index.isin(test_set.index)]

In [12]:
overlap = [i for i in train_set.index if i in val_set.index or i in test_set.index]
print(bool(overlap), len(overlap))

False 0


In [13]:
val_set[['dG','target_loc','ligand_loc']].to_csv('../data/valid.dat', header=False, index=False, sep=' ')
test_set[['dG','target_loc','ligand_loc']].to_csv('../data/test.dat', header=False, index=False, sep=' ')
train_set[['dG','target_loc','ligand_loc']].to_csv('../data/train.dat', header=False, index=False, sep=' ')